In [1]:
import json
from crf_feature_engineering import note2features, note2labels, note2features_withlemma, note2labels_withlemma
from sklearn.model_selection import train_test_split
import sklearn_crfsuite
import sklearn
import pandas as pd
from sklearn_crfsuite import metrics
import eli5
import imgkit

/Users/emmavr/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
# NOTE: Jupyter notebook used to present results visually in ELI5

# check sklearn version <0.24
print(sklearn.__version__)

0.22.2


In [3]:
# read the files
with open('crfsuite_data/crfsuite_data_lemma_both_sessions.json') as f:
    crfsuite_data_lemma_both_sessions = [list(x) for x in json.load(f)]

with open('crfsuite_data/crfsuite_data_no_lemma_both_sessions.json') as f:
    crfsuite_data_no_lemma_both_sessions = [list(x) for x in json.load(f)]

print("Number of notes: ", len(crfsuite_data_no_lemma_both_sessions))

Notater session 1:  62
Notater session 5:  183
Notater begge:  249


In [4]:
X = [note2features_withlemma(n) for n in crfsuite_data_lemma_both_sessions]
y = [note2labels_withlemma(n) for n in crfsuite_data_lemma_both_sessions]

In [5]:
# Split train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

In [6]:
# Create a CRF model with sklearn_crfsuite
# Using L-BFGS training algorithm (it is default) with Elastic Net (L1 + L2) regularization.
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)

In [7]:
# Train the CRF model using the training data
crf.fit(X_train, y_train)

/Users/emmavr/.local/lib/python3.8/site-packages/sklearn/base.py:193: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [8]:
### Evaluation ###

# remove O from labels when calculating metrics since it is very overrepresented
labels = list(crf.classes_)
labels.remove('O')

y_pred = crf.predict(X_test)

#metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=labels)

# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)

print("\nReport:\n")

#print(metrics.flat_classification_report(y_test, y_pred, labels=sorted_labels))

report_dict = metrics.flat_classification_report(y_test, y_pred, labels=sorted_labels, output_dict=True)
report_df = pd.DataFrame(report_dict).T
report_df = report_df.round(decimals=2)
report_df["support"] = report_df["support"].round().astype(int)

display(report_df)



Report:

                        precision    recall  f1-score   support

         B-IV_generelt       0.75      0.62      0.68        24
         I-IV_generelt       0.27      0.44      0.33         9
             B-Pasient       0.73      0.79      0.76        94
             I-Pasient       0.50      0.25      0.33         4
B-Perifert_venekateter       0.85      0.79      0.81        28
I-Perifert_venekateter       0.78      0.88      0.82         8
        B-Tegn_Symptom       0.83      0.75      0.79        57
        I-Tegn_Symptom       0.17      0.12      0.14         8
         B-Vitalt_Tegn       0.83      1.00      0.91         5
         I-Vitalt_Tegn       0.67      1.00      0.80         2

             micro avg       0.72      0.73      0.73       239
             macro avg       0.64      0.66      0.64       239
          weighted avg       0.73      0.73      0.73       239



,precision,recall,f1-score,support
B-IV_generelt,0.75,0.62,0.68,24
I-IV_generelt,0.27,0.44,0.33,9
B-Pasient,0.73,0.79,0.76,94
I-Pasient,0.50,0.25,0.33,4
B-Perifert_venekateter,0.85,0.79,0.81,28
I-Perifert_venekateter,0.78,0.88,0.82,8
B-Tegn_Symptom,0.83,0.75,0.79,57
I-Tegn_Symptom,0.17,0.12,0.14,8
B-Vitalt_Tegn,0.83,1.00,0.91,5
I-Vitalt_Tegn,0.67,1.00,0.80,2


In [9]:
# Save dataframe to csv to add into latex as a table
#report_df.to_csv('results_csv/both_sessions_lemma_classification_report.csv')

In [10]:
eli5.show_weights(crf, top=10)

/Users/emmavr/.local/lib/python3.8/site-packages/sklearn/base.py:193: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


From \ To,O,B-IV_generelt,I-IV_generelt,B-Pasient,I-Pasient,B-Perifert_venekateter,I-Perifert_venekateter,B-Tegn_Symptom,I-Tegn_Symptom,B-Vitalt_Tegn,I-Vitalt_Tegn
O,2.645,0.893,-3.992,0.746,-2.699,0.557,-2.749,0.839,-3.391,-0.012,-2.829
B-IV_generelt,0.269,-0.206,3.367,-1.872,-0.125,-0.658,-0.007,-0.413,-0.389,-0.15,-0.475
I-IV_generelt,-0.074,-0.118,4.23,-1.366,0.0,0.0,0.0,-0.369,-0.114,0.0,-0.279
B-Pasient,0.253,-0.011,-0.661,-0.941,3.07,0.0,-0.127,-0.273,-0.87,0.0,-0.661
I-Pasient,0.0,0.0,0.0,-0.703,1.622,0.0,0.0,-0.127,0.0,0.0,0.0
B-Perifert_venekateter,-0.329,-0.159,-0.35,0.0,0.0,-0.025,3.489,-0.187,-0.013,0.0,-0.054
I-Perifert_venekateter,-0.027,0.0,-0.001,0.0,0.0,0.0,2.085,-0.204,0.0,0.0,0.0
B-Tegn_Symptom,0.019,-0.356,-0.818,-0.434,-0.451,-0.31,-0.573,0.0,3.657,-0.123,-0.62
I-Tegn_Symptom,-0.049,-0.018,-0.087,0.0,0.0,0.0,0.0,-0.035,2.812,0.0,0.0
B-Vitalt_Tegn,-1.118,-0.238,-0.492,0.0,0.0,-0.096,0.0,-0.201,-0.242,0.103,3.502


In [11]:
#expl = eli5.explain_weights(crf, top=10)
#df = eli5.format_as_dataframes(expl)
#display(df)

In [12]:
"""
eli5_report = eli5.show_weights(crf, top=10).data

with open('eli5_both_with_lemma.html', 'w') as f:
    f.write(eli5_report)
"""

"\neli5_report = eli5.show_weights(crf, top=10).data\n\nwith open('eli5_both_with_lemma.html', 'w') as f:\n    f.write(eli5_report)\n"

In [13]:
"""
options = {
    'format': 'png',
    'encoding': "UTF-8",
}

imgkit.from_file('eli5_both_with_lemma.html', 'eli5_both_with_lemma.png', options=options)
"""


'\noptions = {\n    \'format\': \'png\',\n    \'encoding\': "UTF-8",\n}\n\nimgkit.from_file(\'eli5_both_with_lemma.html\', \'eli5_both_with_lemma.png\', options=options)\n'